In [11]:
from mealpy.evolutionary_based import GA
import os
import numpy as np
from mealpy import PermutationVar, BinaryVar, FloatVar

In [12]:
parameters = [
    {"epoch": 1000, "pop_size": 100},
]

In [13]:
def read_sets(file_path, pop_size, num_city):
    """
    Reads a file containing ranked sets and returns them as a list of lists.

    Parameters:
    - file_path (str): Path to the file containing the ranked sets.
    - pop_size (int): Expected population size (number of sets).
    - num_city (int): Expected number of cities (elements per set).

    Returns:
    - list: A list of ranked sets, where each set is a list of integers.

    Raises:
    - ValueError: If the file content does not match the expected dimensions.
    """
    sets = []
    with open(file_path, 'r') as file:
        for line in file:
            set = list(map(int, line.strip().split()))
            sets.append(set)

    # Validate the dimensions of the ranked sets
    if len(sets) != pop_size:
        raise ValueError(f"Expected {pop_size} sets, but found {len(sets)} in the file.")

    for set in sets:
        if len(set) != num_city:
            raise ValueError(f"Each set must have {num_city} elements. Found a set with {len(set)} elements.")

    return np.array(sets)

In [14]:
# Number of repetitions
num_trials = 10

In [15]:
def save_history_file(model, file_dir, file_prefix):
    """
    :param model: The optimization model containing the history
    :param file_dir: The directory where the history file will be saved
    :param file_prefix: Prefix for the history filename
    """
    history_file = os.path.join(file_dir, f"{file_prefix}_history.txt")

    with open(history_file, "w") as f:

        # for epoch in enumerate(model.history.list_global_best):
        for epoch in range(0, 1000):
            # fitness = agent.target.fitness, Global Fitness: {fitness}
            # solution = agent.solution, Solution: {solution}

            # Fetch additional metrics
            diversity = model.history.list_diversity[epoch]
            exploration = model.history.list_exploration[epoch]
            exploitation = model.history.list_exploitation[epoch]
            local_fitness = model.history.list_current_best_fit[epoch]
            global_fitness = model.history.list_global_best_fit[epoch]
            runtime = model.history.list_epoch_time[epoch]

            f.write(
                f"Epoch: {epoch+1}, Local Fitness: {local_fitness}, Global Fitness: {global_fitness}, Diversity: {diversity}, \
Exploration: {exploration}, Exploitation: {exploitation}, Runtime: {runtime}\n"
            )

    print(f"History saved to {history_file}")

# <span style="color:blue; font-weight:bold;">F12021</span>


In [16]:
from opfunu.cec_based.cec2021 import F12021

f1 = F12021(10, f_bias=0)

In [17]:
p1 = {
    "bounds": FloatVar(lb=f1.lb, ub=f1.ub),
    "obj_func": f1.evaluate,
    "minmax": "min",
    "name": "F1",
    "save_population": True
}

In [18]:
# Create the problem instance
bounds = FloatVar(lb=f1.lb, ub=f1.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F12021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F12021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(base_starting_solution_dir, trial_name, "Mt19937.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist
    
    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p1, starting_solutions=mt19937)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")


2025/02/20 10:22:29 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: Solving single objective optimization problem.


2025/02/20 10:22:29 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 1, Current best: 6749506377.963209, Global best: 6749506377.963209, Runtime: 0.02017 seconds
2025/02/20 10:22:29 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 2, Current best: 4569643599.782249, Global best: 4569643599.782249, Runtime: 0.02222 seconds
2025/02/20 10:22:29 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 3, Current best: 2951869162.6593037, Global best: 2951869162.6593037, Runtime: 0.02749 seconds
2025/02/20 10:22:29 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 4, Current best: 2893746007.5402784, Global best: 2893746007.5402784, Runtime: 0.02346 seconds
2025/02/20 10:22:29 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 5, Current best: 1398446219.5060709, Global best: 1398446219.5060709, Runtime: 0.01619 seconds
2025/02/20 10:22:29 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: 

History saved to ..\history\ga\CEC2021\F12021\MT19937\trial_1\e=1000p=100\e=1000p=100_history.txt
Trial 1 - Run 1: Epoch=1000, Pop Size=100
Best fitness: 19845.883356283437
Best solution: {'float': array([-47.54055468, -70.44685535, -29.62921863, -58.25013604,
        22.10804169,  60.04143532,  38.53923948,  18.53504453,
        76.70326557, -38.66703568])}


2025/02/20 10:22:50 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 6, Current best: 665993554.0328776, Global best: 665993554.0328776, Runtime: 0.01518 seconds
2025/02/20 10:22:50 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 7, Current best: 522292342.72727734, Global best: 522292342.72727734, Runtime: 0.01928 seconds
2025/02/20 10:22:50 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 8, Current best: 493961215.4623051, Global best: 493961215.4623051, Runtime: 0.01735 seconds
2025/02/20 10:22:50 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 9, Current best: 316542159.9209815, Global best: 316542159.9209815, Runtime: 0.02241 seconds
2025/02/20 10:22:50 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 10, Current best: 316542159.9209815, Global best: 316542159.9209815, Runtime: 0.02222 seconds
2025/02/20 10:22:50 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1,

History saved to ..\history\ga\CEC2021\F12021\MT19937\trial_2\e=1000p=100\e=1000p=100_history.txt
Trial 2 - Run 1: Epoch=1000, Pop Size=100
Best fitness: 128988.62277406425
Best solution: {'float': array([-67.76243176, -70.47800287, -29.46582257, -58.34419138,
        21.96709413,  59.91876391,  17.73700698,  18.54957852,
        76.48644378, -21.56627713])}


...

2025/02/20 10:23:52 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 6, Current best: 790696505.1844281, Global best: 790696505.1844281, Runtime: 0.01529 seconds
2025/02/20 10:23:52 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 7, Current best: 675323796.6609151, Global best: 675323796.6609151, Runtime: 0.01681 seconds
2025/02/20 10:23:52 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 8, Current best: 294174228.15948325, Global best: 294174228.15948325, Runtime: 0.01681 seconds
2025/02/20 10:23:52 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 9, Current best: 179917212.8896706, Global best: 179917212.8896706, Runtime: 0.01621 seconds
2025/02/20 10:23:52 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 10, Current best: 179917212.8896706, Global best: 179917212.8896706, Runtime: 0.01649 seconds
2025/02/20 10:23:52 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1,

History saved to ..\history\ga\CEC2021\F12021\MT19937\trial_5\e=1000p=100\e=1000p=100_history.txt
Trial 5 - Run 1: Epoch=1000, Pop Size=100
Best fitness: 71425.17309490271
Best solution: {'float': array([-64.19507565, -70.51250262, -29.62226909, -58.3062571 ,
        22.04994276,  59.9255438 ,  21.39852952,  18.65295064,
        76.5630537 , -24.62208094])}


2025/02/20 10:24:12 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 4, Current best: 1761654337.4012585, Global best: 1761654337.4012585, Runtime: 0.02368 seconds
2025/02/20 10:24:12 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 5, Current best: 899484444.4252766, Global best: 899484444.4252766, Runtime: 0.02024 seconds
2025/02/20 10:24:12 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 6, Current best: 899484444.4252766, Global best: 899484444.4252766, Runtime: 0.01829 seconds
2025/02/20 10:24:12 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 7, Current best: 899484444.4252766, Global best: 899484444.4252766, Runtime: 0.01972 seconds
2025/02/20 10:24:12 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 8, Current best: 661319268.4366592, Global best: 661319268.4366592, Runtime: 0.01680 seconds
2025/02/20 10:24:12 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, 

History saved to ..\history\ga\CEC2021\F12021\MT19937\trial_6\e=1000p=100\e=1000p=100_history.txt
Trial 6 - Run 1: Epoch=1000, Pop Size=100
Best fitness: 73786.98640660383
Best solution: {'float': array([-98.1771442 , -70.41881952, -29.64054811, -58.26553833,
        22.12187999,  59.85802733, -13.49018327,  18.66554013,
        76.5695548 ,   4.01135926])}


2025/02/20 10:24:31 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 6, Current best: 2800294476.249276, Global best: 2800294476.249276, Runtime: 0.02479 seconds
2025/02/20 10:24:31 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 7, Current best: 2283025339.3315263, Global best: 2283025339.3315263, Runtime: 0.01950 seconds
2025/02/20 10:24:31 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 8, Current best: 1977056087.971647, Global best: 1977056087.971647, Runtime: 0.02646 seconds
2025/02/20 10:24:31 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 9, Current best: 1755823653.763985, Global best: 1755823653.763985, Runtime: 0.01766 seconds
2025/02/20 10:24:31 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F1, Epoch: 10, Current best: 1352523603.1953907, Global best: 1352523603.1953907, Runtime: 0.01569 seconds
2025/02/20 10:24:31 AM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: F

#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F12021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F12021", "QuasiRandom")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(base_starting_solution_dir, trial_name, "QuasiRandom.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist
    
    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p1, starting_solutions=QuasiRandom)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F12021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F12021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist
    
    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p1, starting_solutions=IBM)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F12021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F12021", "BeamSplitter")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(base_starting_solution_dir, trial_name, "BeamSplitter.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist
    
    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p1, starting_solutions=BeamSplitter)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


# <span style="color:blue; font-weight:bold;">F22021</span>


In [ ]:
from opfunu.cec_based.cec2021 import F22021

f2 = F22021(10, f_bias=0)

In [ ]:
p2 = {
    "bounds": FloatVar(lb=f2.lb, ub=f2.ub),
    "obj_func": f2.evaluate,
    "minmax": "min",
    "name": "F2",
    "save_population": True
}

In [ ]:
# Create the problem instance
bounds = FloatVar(lb=f2.lb, ub=f2.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F22021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F22021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(base_starting_solution_dir, trial_name, "Mt19937.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist
    
    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p2, starting_solutions=mt19937)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F22021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F22021", "QuasiRandom")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(base_starting_solution_dir, trial_name, "QuasiRandom.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist
    
    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p2, starting_solutions=QuasiRandom)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F22021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F22021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist
    
    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p2, starting_solutions=IBM)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F22021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F22021", "BeamSplitter")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(base_starting_solution_dir, trial_name, "BeamSplitter.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist
    
    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p2, starting_solutions=BeamSplitter)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


# <span style="color:blue; font-weight:bold;">F32021</span>


In [ ]:
from opfunu.cec_based.cec2021 import F32021

f3 = F32021(10, f_bias=0)

In [ ]:
p3 = {
    "bounds": FloatVar(lb=f3.lb, ub=f3.ub),
    "obj_func": f3.evaluate,
    "minmax": "min",
    "name": "F3",
    "save_population": True
}

In [ ]:
# Create the problem instance
bounds = FloatVar(lb=f3.lb, ub=f3.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F32021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F32021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(base_starting_solution_dir, trial_name, "Mt19937.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist
    
    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p3, starting_solutions=mt19937)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F32021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F32021", "QuasiRandom")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(base_starting_solution_dir, trial_name, "QuasiRandom.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist
    
    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p3, starting_solutions=QuasiRandom)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F32021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F32021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist
    
    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p3, starting_solutions=IBM)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F32021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F32021", "BeamSplitter")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(base_starting_solution_dir, trial_name, "BeamSplitter.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist
    
    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p3, starting_solutions=BeamSplitter)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


# <span style="color:blue; font-weight:bold;">F42021</span>


In [ ]:
from opfunu.cec_based.cec2021 import F42021

f4 = F42021(10, f_bias=0)

In [ ]:
p4 = {
    "bounds": FloatVar(lb=f4.lb, ub=f4.ub),
    "obj_func": f4.evaluate,
    "minmax": "min",
    "name": "F4",
    "save_population": True
}

In [ ]:
# Create the problem instance
bounds = FloatVar(lb=f4.lb, ub=f4.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F42021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F42021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(base_starting_solution_dir, trial_name, "Mt19937.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist
    
    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p4, starting_solutions=mt19937)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F42021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F42021", "QuasiRandom")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(base_starting_solution_dir, trial_name, "QuasiRandom.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist
    
    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p4, starting_solutions=QuasiRandom)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F42021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F42021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist
    
    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p4, starting_solutions=IBM)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F42021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F42021", "BeamSplitter")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(base_starting_solution_dir, trial_name, "BeamSplitter.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist
    
    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p4, starting_solutions=BeamSplitter)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


# <span style="color:blue; font-weight:bold;">F52021</span>


In [ ]:
from opfunu.cec_based.cec2021 import F52021

f5 = F52021(10, f_bias=0)

In [ ]:
p5 = {
    "bounds": FloatVar(lb=f5.lb, ub=f5.ub),
    "obj_func": f5.evaluate,
    "minmax": "min",
    "name": "F5",
    "save_population": True
}

In [ ]:
# Create the problem instance
bounds = FloatVar(lb=f5.lb, ub=f5.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F52021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F52021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(base_starting_solution_dir, trial_name, "Mt19937.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist
    
    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p5, starting_solutions=mt19937)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F52021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F52021", "QuasiRandom")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(base_starting_solution_dir, trial_name, "QuasiRandom.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist
    
    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p5, starting_solutions=QuasiRandom)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F52021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F52021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist
    
    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p5, starting_solutions=IBM)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F52021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F52021", "BeamSplitter")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(base_starting_solution_dir, trial_name, "BeamSplitter.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist
    
    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p5, starting_solutions=BeamSplitter)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


# <span style="color:blue; font-weight:bold;">F62021</span>


In [ ]:
from opfunu.cec_based.cec2021 import F62021

f6 = F62021(10, f_bias=0)

In [ ]:
p6 = {
    "bounds": FloatVar(lb=f6.lb, ub=f6.ub),
    "obj_func": f6.evaluate,
    "minmax": "min",
    "name": "F6",
    "save_population": True
}

In [ ]:
# Create the problem instance
bounds = FloatVar(lb=f6.lb, ub=f6.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F62021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F62021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(base_starting_solution_dir, trial_name, "Mt19937.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist
    
    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p6, starting_solutions=mt19937)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F62021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F62021", "QuasiRandom")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(base_starting_solution_dir, trial_name, "QuasiRandom.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist
    
    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p6, starting_solutions=QuasiRandom)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F62021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F62021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist
    
    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p6, starting_solutions=IBM)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F62021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F62021", "BeamSplitter")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(base_starting_solution_dir, trial_name, "BeamSplitter.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist
    
    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p6, starting_solutions=BeamSplitter)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


# <span style="color:blue; font-weight:bold;">F72021</span>


In [ ]:
from opfunu.cec_based.cec2021 import F72021

f7 = F72021(10, f_bias=0)

In [ ]:
p7 = {
    "bounds": FloatVar(lb=f7.lb, ub=f7.ub),
    "obj_func": f7.evaluate,
    "minmax": "min",
    "name": "F7",
    "save_population": True
}

In [ ]:
# Create the problem instance
bounds = FloatVar(lb=f7.lb, ub=f7.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F72021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F72021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(base_starting_solution_dir, trial_name, "Mt19937.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist
    
    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p7, starting_solutions=mt19937)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F72021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F72021", "QuasiRandom")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(base_starting_solution_dir, trial_name, "QuasiRandom.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist
    
    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p7, starting_solutions=QuasiRandom)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F72021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F72021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist
    
    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p7, starting_solutions=IBM)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F72021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F72021", "BeamSplitter")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(base_starting_solution_dir, trial_name, "BeamSplitter.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist
    
    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p7, starting_solutions=BeamSplitter)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


# <span style="color:blue; font-weight:bold;">F82021</span>


In [ ]:
from opfunu.cec_based.cec2021 import F82021

f8 = F82021(10, f_bias=0)

In [ ]:
p8 = {
    "bounds": FloatVar(lb=f8.lb, ub=f8.ub),
    "obj_func": f8.evaluate,
    "minmax": "min",
    "name": "F8",
    "save_population": True
}

In [ ]:
# Create the problem instance
bounds = FloatVar(lb=f8.lb, ub=f8.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F82021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F82021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(base_starting_solution_dir, trial_name, "Mt19937.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist
    
    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p8, starting_solutions=mt19937)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F82021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F82021", "QuasiRandom")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(base_starting_solution_dir, trial_name, "QuasiRandom.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist
    
    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p8, starting_solutions=QuasiRandom)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F82021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F82021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist
    
    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p8, starting_solutions=IBM)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F82021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F82021", "BeamSplitter")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(base_starting_solution_dir, trial_name, "BeamSplitter.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist
    
    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p8, starting_solutions=BeamSplitter)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


# <span style="color:blue; font-weight:bold;">F92021</span>


In [ ]:
from opfunu.cec_based.cec2021 import F92021

f9 = F92021(10, f_bias=0)

In [ ]:
p9 = {
    "bounds": FloatVar(lb=f9.lb, ub=f9.ub),
    "obj_func": f9.evaluate,
    "minmax": "min",
    "name": "F9",
    "save_population": True
}

In [ ]:
# Create the problem instance
bounds = FloatVar(lb=f9.lb, ub=f9.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F92021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F92021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(base_starting_solution_dir, trial_name, "Mt19937.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist
    
    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p9, starting_solutions=mt19937)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F92021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F92021", "QuasiRandom")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(base_starting_solution_dir, trial_name, "QuasiRandom.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist
    
    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p9, starting_solutions=QuasiRandom)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F92021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F92021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist
    
    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p9, starting_solutions=IBM)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F92021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F92021", "BeamSplitter")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(base_starting_solution_dir, trial_name, "BeamSplitter.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist
    
    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p9, starting_solutions=BeamSplitter)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


# <span style="color:blue; font-weight:bold;">F102021</span>


In [ ]:
from opfunu.cec_based.cec2021 import F102021

f10 = F102021(10, f_bias=0)

In [ ]:
p10 = {
    "bounds": FloatVar(lb=f10.lb, ub=f10.ub),
    "obj_func": f10.evaluate,
    "minmax": "min",
    "name": "F10",
    "save_population": True
}

In [ ]:
# Create the problem instance
bounds = FloatVar(lb=f10.lb, ub=f10.ub)

#### <span style="color:green; font-weight:bold;">-Mersene Twister</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F102021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F102021", "MT19937")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_mt19937 = os.path.join(base_starting_solution_dir, trial_name, "Mt19937.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_mt19937):
        print(f"Skipping {file_path_mt19937} (File not found)")
        continue  # Skip if the file does not exist
    
    mt19937 = read_sets(file_path_mt19937, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p10, starting_solutions=mt19937)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Quasirandom</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F102021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F102021", "QuasiRandom")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_QuasiRandom = os.path.join(base_starting_solution_dir, trial_name, "QuasiRandom.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_QuasiRandom):
        print(f"Skipping {file_path_QuasiRandom} (File not found)")
        continue  # Skip if the file does not exist
    
    QuasiRandom = read_sets(file_path_QuasiRandom, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p10, starting_solutions=QuasiRandom)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-IBM qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F102021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F102021", "IBM")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_IBM = os.path.join(base_starting_solution_dir, trial_name, "IBM.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_IBM):
        print(f"Skipping {file_path_IBM} (File not found)")
        continue  # Skip if the file does not exist
    
    IBM = read_sets(file_path_IBM, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p10, starting_solutions=IBM)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        


#### <span style="color:green; font-weight:bold;">-Beam splitter qrng</span>


In [ ]:
# Base directories
base_starting_solution_dir = os.path.join("..", "starting_solution", "CEC2021", "F102021")
base_history_dir = os.path.join("..", "history", "ga", "CEC2021", "F102021", "BeamSplitter")

# Ensure the base history directory exists
os.makedirs(base_history_dir, exist_ok=True)

# Loop through each trial directory (trial_1 to trial_9)
for trial_num in range(1, num_trials + 1):
    # Construct the input file path for the current trial
    trial_name = f"trial_{trial_num}"
    file_path_BeamSplitter = os.path.join(base_starting_solution_dir, trial_name, "BeamSplitter.txt")
    
    # Read the corresponding starting solution
    if not os.path.exists(file_path_BeamSplitter):
        print(f"Skipping {file_path_BeamSplitter} (File not found)")
        continue  # Skip if the file does not exist
    
    BeamSplitter = read_sets(file_path_BeamSplitter, 100, 10)

    # Define the history directory for this trial
    trial_history_dir = os.path.join(base_history_dir, trial_name)
    os.makedirs(trial_history_dir, exist_ok=True)  # Ensure the directory exists

    # Loop through the parameter sets
    for idx, params in enumerate(parameters, start=1):
        # Generate a sub-directory and filename prefix based on epoch and population size
        file_prefix = f"e={params['epoch']}p={params['pop_size']}"
        file_dir = os.path.join(trial_history_dir, file_prefix)
        os.makedirs(file_dir, exist_ok=True)  # Ensure the directory exists

        # Initialize and solve the model with specific parameters
        model = GA.BaseGA(epoch=params["epoch"], pop_size=params["pop_size"])
        history = model.solve(p10, starting_solutions=BeamSplitter)
        
        # Save the optimization history to a text file
        save_history_file(model, file_dir, file_prefix)
        
        # Print the best results for each run
        print(f"Trial {trial_num} - Run {idx}: Epoch={params['epoch']}, Pop Size={params['pop_size']}")
        print(f"Best fitness: {model.g_best.target.fitness}")
        print(f"Best solution: {model.problem.decode_solution(model.g_best.solution)}")
        
